In [1]:
import datasets
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from random import sample
from itertools import combinations

In [2]:
stance_list = ["stance_abortion","stance_atheism", "stance_climate","stance_feminist", "stance_hillary"]

In [3]:
# # tweet_eval 생성시
# #task_name = stance_list[4] # "emoji"
# task_name = "emotion"
# num_label = 4
# # label 별로 생성 상수, 2->1, 3->1.5 4->2
# emoji_dataset = datasets.load_dataset("tweet_eval", task_name)
# emoji_dataset

In [4]:
# # sst2생성시
# num_label = 2
# task_name="sst2"
# # label 별로 생성 상수, 2->1, 3->1.5 4->2
# sst2 = load_dataset(task_name)
# sst2 = sst2.remove_columns(["idx"])
# sst2 = sst2.rename_column('sentence', 'text')
# emoji_dataset = sst2
# emoji_dataset

In [5]:
# # sst5생성시
# num_label = 5
# task_name="sst5"
# # label 별로 생성 상수, 2->1, 3->1.5 4->2
# sst5 = load_dataset("SetFit/"+task_name)
# sst5 = sst5.remove_columns(["label_text"])
# emoji_dataset = sst5
# emoji_dataset

/home/uj-user/miniconda3/envs/simcse/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Using custom data configuration SetFit--sst5-0c891184cb873f87
Reusing dataset json (/home/uj-user/.cache/huggingface/datasets/SetFit___json/SetFit--sst5-0c891184cb873f87/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8544
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2210
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1101
    })
})

In [17]:
# tweet_eval 생성시
#task_name = stance_list[4] # "emoji"
task_name = "yelp_polarity"
num_label = 2
# label 별로 생성 상수, 2->1, 3->1.5 4->2
emoji_dataset = datasets.load_dataset(task_name)
emoji_dataset

Reusing dataset yelp_polarity (/home/uj-user/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})

In [18]:
# emoji_train = emoji_dataset['train'].to_pandas()
# emoji_valid = emoji_dataset['validation'].to_pandas()
# emoji_test = emoji_dataset['test'].to_pandas()
emoji_valid = emoji_dataset['validation'].to_pandas()
emoji_valid['label'].value_counts()

KeyError: 'validation'

In [7]:
smallest_label_num = sorted(emoji_valid['label'].value_counts().tolist())[0]
smallest_label_num

139

# 데이터셋 생성함수

In [8]:
sts_emoji_pair_list = []

def make_same_pair(label_num, data_num, seed=2022):
    global emoji_valid
    # 1. 같은 레이블끼리의 데이터를 2개뽑아 1또는 2로 지정
    for i in range(label_num): # 레이블별로 
        for j in range(data_num): # 300개 추출
            pair = emoji_valid[emoji_valid['label']==i].sample(2, random_state=seed)
            emoji_valid = emoji_valid.drop(pair.index) # 추출된 데이터 삭제

            pair_output = pair["text"].tolist()
            sts_emoji_pair_list.append([pair_output[0], pair_output[1], 1]) # 레이블1 데이터 추가

def make_another_pair(label_num, data_num, seed=2022):
    global emoji_valid
    # 2. 각기 다른 레이블을 두개 뽑아서 0으로 지정
    all_group = list(range(label_num))
    a = 0
    for a_label, b_label in list(combinations(all_group, 2)): # 20C2 = 190개의 조합 
        for _ in range(data_num):
            a_pair = emoji_valid[emoji_valid['label']==a_label].sample(1, random_state=seed)
            b_pair = emoji_valid[emoji_valid['label']==b_label].sample(1, random_state=seed)
            
            emoji_valid = emoji_valid.drop([a_pair.index[0], b_pair.index[0]])
            pair_output = [a_pair["text"].tolist()[0], b_pair["text"].tolist()[0], 0]
            sts_emoji_pair_list.append(pair_output)
            a = a + 1
            #print(a)


def make_group_pair(label_num, data_num, exculsive_group, seed=2022):
    global emoji_valid
    # 3. 각기 다른 레이블 0으로 지정
    all_group = list(range(20))
    for a_label, b_label in list(combinations(all_group, 2)): # 20C2 = 190개의 조합
        for _ in range(data_num):
            a_pair = emoji_valid[emoji_valid['label']==a_label].sample(1, random_state=seed)
            b_pair = emoji_valid[emoji_valid['label']==b_label].sample(1, random_state=seed)
            
            emoji_valid = emoji_valid.drop([a_pair.index[0], b_pair.index[0]])
            pair_output = [a_pair["text"].tolist()[0], b_pair["text"].tolist()[0], 0]
            sts_emoji_pair_list.append(pair_output)

In [9]:
# 2->4, 3->4, 4->4, 5->?

In [10]:
emoji_valid = emoji_dataset['validation'].to_pandas() # 데이터불러오기 #함수실행시 데이터 줄어듬
# print(emoji_valid['label'].value_counts())
sts_emoji_pair_list = [] # 담을 리스트

make_same_pair(num_label, int(smallest_label_num//4)) # 20 * 300
print("생성데이터갯수:", len(sts_emoji_pair_list), " 남은데이터:", emoji_valid['label'].count())
print(emoji_valid['label'].value_counts())

생성데이터갯수: 170  남은데이터: 761
1    221
3    211
2    161
4     97
0     71
Name: label, dtype: int64


In [11]:
from math import comb
remain_label_num = sorted(emoji_valid['label'].value_counts().tolist())[0]
remain_label_num
each_iter_generated_num = comb(num_label,2)
print(remain_label_num, each_iter_generated_num)

71 10


In [12]:
# 계산식이 이상해보이겠지만. 최대한 레이블갯수가 비슷하도록 생성함
# # 최대생성가능갯수: 가장작은 레이블의 데이터 갯수 / (레이블갯수 Combination 2)
# 2->1 3->0.75 4->0.5, 5->0.4

make_another_pair(num_label, int(remain_label_num // (each_iter_generated_num*2/num_label))) 
print("생성데이터갯수:", len(sts_emoji_pair_list), " 남은데이터:", emoji_valid['label'].count())
print(emoji_valid['label'].value_counts())

생성데이터갯수: 340  남은데이터: 421
1    153
3    143
2     93
4     29
0      3
Name: label, dtype: int64


In [13]:
df_emoji_pair = pd.DataFrame(sts_emoji_pair_list, columns=["text1", "text2", "label"])
df_emoji_pair

,text1,text2,label
0,"this piece of channel 5 grade trash is , quite...",what is 100 % missing here is a script of even...,1
1,so unremittingly awful that labeling it a dog ...,"the humor is n't as sharp , the effects not as...",1
2,i do n't think i laughed out loud once .,the affectionate loopiness that once seemed co...,1
3,"yes , dull .",taylor appears to have blown his entire budget...,1
4,"green might want to hang onto that ski mask , ...",so devoid of any kind of intelligible story th...,1
...,...,...,...
335,a movie that reminds us of just how exciting a...,"one of those energetic surprises , an original...",0
336,zhang ... has done an amazing job of getting r...,the film 's performances are thrilling .,0
337,the sort of film that makes me miss hitchcock ...,it inspires a continuing and deeply satisfying...,0
338,"the jabs it employs are short , carefully plac...","altogether , this is successful as a film , wh...",0


In [14]:
df_emoji_pair['label'].value_counts()

1    170
0    170
Name: label, dtype: int64

In [15]:
fpath ="/home/uj-user/SenCSE/SentEval/data/downstream/TEMP"
df_emoji_pair.to_csv(f'{fpath}/TWEET.input.{task_name}.csv', index=False)

In [16]:
# for stance last stance 4 reaches then runned.
if task_name in stance_list[4]:
    stance_list = ["stance_abortion","stance_atheism", "stance_climate","stance_feminist", "stance_hillary"]

    df = pd.DataFrame(columns=['text1', 'text2', 'label'])
    for stance_name in stance_list:
        new_df = pd.read_csv(f'{fpath}/TWEET.input.{stance_name}.csv')
        print(new_df['label'].value_counts())
        df = pd.concat([df, new_df])

    df.to_csv(f'{fpath}/TWEET.input.stance.csv', index=False)